# KPI Definition & Validation — Online Retail (2010–2011)

This notebook focuses on **metric definition** and **data-quality validation** before any time-series analysis.

## Why this matters
In analytics work, trends can be misleading if:
- KPI numerator/denominator is unclear
- returns/cancellations are mixed into “orders” or “revenue”
- missing customer identifiers distort customer metrics

## Outputs of this notebook
We will produce a validated KPI table `daily_validated` with:
- revenue definitions: **gross_sales** vs **net_revenue**
- order definitions: **orders_all** vs **orders_non_cancel**
- quality indicators: **return_rate**, **cancel_share**, **unknown_customer_row_share**

> Interview one-liner:  
> **"Before analyzing trends, I first validate the metric definition."**

In [1]:
import pandas as pd
import numpy as np
import os

# Daily KPI table exported from Notebook 01 (data preparation)
DAILY_KPI_PATH = "data/daily_kpi.csv"

# Raw CSV path (temporary: project root). 
# Later, move it to data/online_retail_raw.csv and update this variable.
RAW_PATH = "online_retail.csv"

print("Daily KPI path:", DAILY_KPI_PATH)
print("Raw path:", RAW_PATH)

Daily KPI path: data/daily_kpi.csv
Raw path: online_retail.csv


## Step 0 — Load the daily KPI table (from Notebook 01)

We start from the **daily aggregated KPI table** to avoid re-computing everything from raw.
Raw data will be used only for **validation** (cancellations, missing CustomerID).

In [2]:
daily_kpi = pd.read_csv(DAILY_KPI_PATH)
daily_kpi["date"] = pd.to_datetime(daily_kpi["date"])
daily_kpi = daily_kpi.sort_values("date")

print("daily_kpi shape:", daily_kpi.shape)
print("date range:", daily_kpi["date"].min(), "to", daily_kpi["date"].max())
daily_kpi.head()

daily_kpi shape: (280, 8)
date range: 2010-12-01 00:00:00 to 2011-11-10 00:00:00


,date,orders,active_customers,gross_revenue,return_revenue,items_sold,rows,net_revenue
0,2010-12-01,143,98,58635.56,-325.23,27007.0,3108,58635.56
1,2010-12-02,167,117,46207.28,-1541.10,31348.0,2109,46207.28
2,2010-12-03,108,55,45620.46,-1323.25,16471.0,2202,45620.46
3,2010-12-05,95,76,31383.95,-391.00,16451.0,2725,31383.95
4,2010-12-06,133,90,53860.18,-970.28,21951.0,3878,53860.18


## Step 1 — Revenue metric definitions (Gross Sales vs Net Revenue)

In this dataset, line-level revenue is computed as:

- `line_revenue = Quantity × UnitPrice`
- returns usually have negative `Quantity`, so aggregated revenue naturally includes returns.

Therefore:
- **Net Revenue**: realized revenue after returns → from the prep table `gross_revenue` (name is misleading)
- **Gross Sales**: sales volume without subtracting returns  
  = `net_revenue - return_revenue` (because return_revenue is negative)

We also define:
- **Return Rate** = `(-return_revenue) / gross_sales`

In [3]:
daily = daily_kpi.copy()

# Naming fix: gross_revenue from prep table should be interpreted as net_revenue
daily["net_revenue"] = daily["gross_revenue"]

# Gross sales = add back returns (return_revenue is negative)
daily["gross_sales"] = daily["net_revenue"] - daily["return_revenue"]

# Return rate (use gross_sales as denominator)
daily["return_rate"] = np.where(
    daily["gross_sales"] != 0,
    (-daily["return_revenue"]) / daily["gross_sales"],
    np.nan
)

daily[["date", "gross_sales", "net_revenue", "return_revenue", "return_rate"]].head()

,date,gross_sales,net_revenue,return_revenue,return_rate
0,2010-12-01,58960.79,58635.56,-325.23,0.005516
1,2010-12-02,47748.38,46207.28,-1541.10,0.032275
2,2010-12-03,46943.71,45620.46,-1323.25,0.028188
3,2010-12-05,31774.95,31383.95,-391.00,0.012305
4,2010-12-06,54830.46,53860.18,-970.28,0.017696


In [4]:
daily[["gross_sales", "net_revenue", "return_rate"]].describe()

,gross_sales,net_revenue,return_rate
count,280.000000,280.000000,280.000000
mean,32169.474800,29772.337050,0.058417
std,17120.660822,15448.170485,0.121760
min,3457.110000,-1566.230000,-0.000000
25%,21808.400000,20147.207500,0.006733
50%,28342.015500,26655.425000,0.017383
75%,40496.932500,37320.635000,0.053506
max,109612.030000,109286.210000,1.047993


## Step 2 — Raw-data validation (cancellations & missing CustomerID)

We use raw data only to validate:
1. **Cancellations**: invoices where `InvoiceNo` starts with `"C"`
2. **CustomerID missingness**: share of rows without a valid CustomerID

Important: For data-quality shares, we compute numerator and denominator from the **same raw table**
to avoid inconsistencies due to different cleaning pipelines.

In [5]:
raw = pd.read_csv(RAW_PATH, encoding="ISO-8859-1")

# Minimal cleaning: datetime
raw["InvoiceDate"] = pd.to_datetime(raw["InvoiceDate"], errors="coerce")
raw = raw.dropna(subset=["InvoiceDate"]).copy()
raw["date"] = raw["InvoiceDate"].dt.date

# Optional: enforce types (for consistency)
raw["InvoiceNo"] = raw["InvoiceNo"].astype(str)
raw["CustomerID"] = pd.to_numeric(raw["CustomerID"], errors="coerce")

print("raw shape (after datetime cleaning):", raw.shape)
print("raw date range:", raw["InvoiceDate"].min(), "to", raw["InvoiceDate"].max())
raw.head()

raw shape (after datetime cleaning): (541909, 9)
raw date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


## Step 3 — Order definition validation (All vs Non-cancel)

We compute daily:
- `orders_all`: all unique invoices
- `orders_non_cancel`: unique invoices excluding cancellations (`InvoiceNo` starts with `"C"`)
- `cancel_orders`: unique cancelled invoices
- `cancel_share`: cancel_orders / orders_all

This avoids fragile groupby-lambda patterns by slicing first, then grouping.

In [6]:
raw["is_cancel"] = raw["InvoiceNo"].str.startswith("C")

orders_all = raw.groupby("date")["InvoiceNo"].nunique()
orders_non_cancel = raw.loc[~raw["is_cancel"]].groupby("date")["InvoiceNo"].nunique()
cancel_orders = raw.loc[raw["is_cancel"]].groupby("date")["InvoiceNo"].nunique()

orders_check = pd.concat([orders_all, orders_non_cancel, cancel_orders], axis=1).reset_index()
orders_check.columns = ["date", "orders_all", "orders_non_cancel", "cancel_orders"]
orders_check = orders_check.fillna(0)

orders_check["date"] = pd.to_datetime(orders_check["date"])
orders_check = orders_check.sort_values("date")

orders_check["cancel_share"] = np.where(
    orders_check["orders_all"] != 0,
    orders_check["cancel_orders"] / orders_check["orders_all"],
    np.nan
)

orders_check.head()

,date,orders_all,orders_non_cancel,cancel_orders,cancel_share
0,2010-12-01,143,137,6.0,0.041958
1,2010-12-02,167,144,23.0,0.137725
2,2010-12-03,108,101,7.0,0.064815
3,2010-12-05,95,88,7.0,0.073684
4,2010-12-06,133,115,18.0,0.135338


## Step 4 — Customer metric validation (known customers vs missing IDs)

Customer metrics depend on whether customers can be identified:
- `active_customers_known`: unique non-null CustomerID per day
- `unknown_customer_rows`: number of rows with missing CustomerID per day
- `unknown_customer_row_share`: unknown_customer_rows / total_rows (raw-based denominator)

In [7]:
cust_check = (
    raw.groupby("date")
       .agg(
           active_customers_known=("CustomerID", lambda s: s.dropna().nunique()),
           unknown_customer_rows=("CustomerID", lambda s: s.isna().sum()),
           total_rows=("CustomerID", "size")
       )
       .reset_index()
)

cust_check["unknown_customer_row_share"] = cust_check["unknown_customer_rows"] / cust_check["total_rows"]
cust_check["date"] = pd.to_datetime(cust_check["date"])
cust_check = cust_check.sort_values("date")

cust_check.head()

,date,active_customers_known,unknown_customer_rows,total_rows,unknown_customer_row_share
0,2010-12-01,98,1140,3108,0.366795
1,2010-12-02,117,65,2109,0.030820
2,2010-12-03,55,1085,2202,0.492734
3,2010-12-05,76,1,2725,0.000367
4,2010-12-06,90,1904,3878,0.490975


## Step 5 — Build the validated daily KPI table

We merge:
- revenue definitions (from daily_kpi)
- order validation results (from raw)
- customer missingness validation results (from raw)

The result is the final table `daily_validated` for time-series analysis.

In [8]:
daily_validated = (
    daily.merge(orders_check[["date", "orders_all", "orders_non_cancel", "cancel_orders", "cancel_share"]],
                on="date", how="left")
         .merge(cust_check[["date", "active_customers_known", "unknown_customer_rows", "unknown_customer_row_share"]],
                on="date", how="left")
)

cols_preview = [
    "date",
    "orders", "orders_all", "orders_non_cancel", "cancel_share",
    "active_customers", "active_customers_known", "unknown_customer_row_share",
    "gross_sales", "net_revenue", "return_rate"
]

daily_validated[cols_preview].head()

,date,orders,orders_all,orders_non_cancel,cancel_share,active_customers,active_customers_known,unknown_customer_row_share,gross_sales,net_revenue,return_rate
0,2010-12-01,143,143,137,0.041958,98,98,0.366795,58960.79,58635.56,0.005516
1,2010-12-02,167,167,144,0.137725,117,117,0.030820,47748.38,46207.28,0.032275
2,2010-12-03,108,108,101,0.064815,55,55,0.492734,46943.71,45620.46,0.028188
3,2010-12-05,95,95,88,0.073684,76,76,0.000367,31774.95,31383.95,0.012305
4,2010-12-06,133,133,115,0.135338,90,90,0.490975,54830.46,53860.18,0.017696


In [10]:
daily_validated[["gross_sales", "net_revenue", "return_rate"]].describe()

,gross_sales,net_revenue,return_rate
count,280.000000,280.000000,280.000000
mean,32169.474800,29772.337050,0.058417
std,17120.660822,15448.170485,0.121760
min,3457.110000,-1566.230000,-0.000000
25%,21808.400000,20147.207500,0.006733
50%,28342.015500,26655.425000,0.017383
75%,40496.932500,37320.635000,0.053506
max,109612.030000,109286.210000,1.047993


In [11]:
daily_validated[["cancel_share", "unknown_customer_row_share"]].describe()

,cancel_share,unknown_customer_row_share
count,280.000000,280.000000
mean,0.143376,0.218837
std,0.083425,0.178535
min,0.000000,0.000000
25%,0.077886,0.062775
50%,0.132585,0.190713
75%,0.193220,0.334615
max,0.463158,0.874498


In [12]:
daily_validated.tail()

,date,orders,active_customers,gross_revenue,return_revenue,items_sold,rows,net_revenue,gross_sales,return_rate,orders_all,orders_non_cancel,cancel_orders,cancel_share,active_customers_known,unknown_customer_rows,unknown_customer_row_share
275,2011-11-06,104,88,42912.40,-33.94,23309.0,3437,42912.40,42946.34,0.000790,104,102,2.0,0.019231,88,3,0.000873
276,2011-11-07,115,90,70001.08,-15880.73,31395.0,2099,70001.08,85881.81,0.184914,115,99,16.0,0.139130,90,323,0.153883
277,2011-11-08,144,122,56647.66,-1052.58,26538.0,4070,56647.66,57700.24,0.018242,144,110,34.0,0.236111,122,1671,0.410565
278,2011-11-09,127,102,62599.43,-3528.14,38269.0,2716,62599.43,66127.57,0.053354,127,121,6.0,0.047244,102,517,0.190353
279,2011-11-10,141,111,54891.21,-2052.02,32019.0,2528,54891.21,56943.23,0.036036,184,148,36.0,0.195652,139,81,0.024733


## KPI Policy (Final)

### Revenue
- We report two revenue metrics:
  - **Gross Sales**: sales volume without subtracting returns  
  - **Net Revenue**: realized revenue after subtracting returns (primary metric for trend analysis)
- We track **Return Rate** = `(-return_revenue) / gross_sales` as a health metric.

### Orders
- We distinguish:
  - **All orders**: all invoices
  - **Non-cancel orders**: invoices excluding cancellations (`InvoiceNo` starts with `"C"`)
- For core performance monitoring, we use **Non-cancel orders**.

### Active Customers
- We count only valid `CustomerID` as **Known Active Customers**.
- Missing `CustomerID` rows cannot be attributed to users, so we track  
  **unknown_customer_row_share** as a data-quality indicator.

## Step 6 — Export validated KPI table

This table will be the only input to the time-series notebook.

In [13]:
os.makedirs("data", exist_ok=True)

out_path = "data/daily_kpi_validated.csv"
daily_validated.to_csv(out_path, index=False)

print("Saved:", out_path)

Saved: data/daily_kpi_validated.csv


In [16]:
import os
os.makedirs("powerbi", exist_ok=True)

daily_kpi.to_csv(
    "powerbi/daily_kpi.csv",
    index=False
)